In [ ]:
import datetime
import webbrowser
import time
import string
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import csv
from tkinter import *
from tkinter import messagebox
import pandas as pd

pd.set_option('display.expand_frame_repr',False)
global User
global Pass
User = ""
Pass = ""     

def netflix_window():
        window1 = Toplevel(root)
        window1.title("Netflix Results")
        df = pd.read_csv("NetflixResults.csv")
        text = Text(window1)
        text.insert(END, str(df))
        text.pack(expand=True, fill=BOTH)
    
    
def amazon_window():
        window2 = Toplevel(root)   
        window2.title("Amazon Results")
        df1 = pd.read_csv("amazonresults.csv")
        text = Text(window2)
        text.insert(END, str(df1))
        text.pack(expand=True, fill=BOTH)    
    
def NetflixLogin():
    global Username
    global password
    window3 = Toplevel(root)
    window3.title("Netflix Login")
    label1 = Label(window3, text = "Username")
    label2 = Label(window3, text = "Password")
    button1=Button(window3, text="Enter!", width=20)
    Username = Entry(window3,width=15)
    password = Entry(window3,show="*", width=15)
    label1.grid(row=1, column=1, padx=5, sticky=W)
    label2.grid(row=4, column=1, padx=5, sticky=W)
    Username.grid(row=2,column=1,padx=5, sticky=W)
    password.grid(row=5,column=1,padx=5, sticky=W)
    button1.grid(row=6,column=1,columnspan=2,pady=5)
    
    button1.configure(command=UserPassStorage)     
    
def UserPassStorage():
    global Username
    global password
    global User
    global Pass
    User=Username.get()
    Pass = password.get()
     
    
    
def resultsDisplay():
        global User
        global Pass
        text = entry.get('1.0', END)  
        searchphrase = string.capwords(text)
        if int(len(searchphrase)) > 0:
            if int(len(User)) and int(len(Pass)) > 0:
                result1 = amazonsearch(searchphrase)  
                result2 = NetflixSearch(searchphrase)  
                output = Text(master=root)  
                output.grid(row=6,column=1,columnspan=5,padx=5,pady=(0,10))          

                output.insert(END, result1)  
                output.insert(END, result2)
            else:
                messagebox.showwarning("Warning", "No Netflix Credentials Found, Performing Amazon Search Only")
                result1 = amazonsearch(searchphrase)
                output = Text(master=root)  
                output.grid(row=6,column=1,columnspan=5,padx=5,pady=(0,10))
                output.insert(END, result1)
        else:
            messagebox.showerror("Error", "You won't find anything with an empty search box you know?")

        
def NetflixSearch(searchphrase):
        global User
        global Pass
        chrome_options = Options()
        #chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920x1080")
        chrome_options.add_argument("--enable-automation")

        driver = webdriver.Chrome(chrome_options=chrome_options, executable_path='chromedriver.exe')
        driver.get("https://www.netflix.com/")
        login = driver.find_element_by_css_selector(".authLinks.signupBasicHeader")
        login.click()
        element = driver.find_element_by_name("email")
        element.send_keys(User)
        element2 = driver.find_element_by_name("password")
        element2.send_keys(Pass)
        submit2 = driver.find_element_by_css_selector(".btn.login-button.btn-submit.btn-small")
        submit2.click()

        profiles = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'profile-icon')))
        profiles.click()

        searchTab = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'icon-search')))
        searchTab.click()

        search = driver.find_element_by_xpath('//*[@id="appMountPoint"]/div/div/div/div[2]/div/div/div/div[1]/div/div/input')
        search.send_keys(searchphrase)

        time.sleep(5)
        title = ""
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        for match in soup.find_all('div', class_= 'title-card', id = 'title-card-0-0' ):
            title = match.get_text

        videourl= ""
        videourl = driver.find_elements_by_xpath('//*[@id="title-card-0-0"]/div/a')
        for ii in videourl:
            video = ii.get_attribute('href')

        my_list = [i.get_text() for i in match]
        my_list = [my_list.replace(',','') for my_list in my_list]
        my_list = [my_list.replace("'",'') for my_list in my_list]
        my_list = [my_list.replace(":",'') for my_list in my_list]
        new_list = list(map(string.capwords, my_list))
        driver.close()
        with open("NetflixResults.csv", "a") as csv_file2:
            writer = csv.writer(csv_file2)
            if new_list[0] == searchphrase:
                result = str(searchphrase + " is on Netflix\n" + "Here is your link\n"  + video + "\nEnjoy!")
                writer.writerow([searchphrase, new_list[0], video,"Sucessfully Found"])
            else:
                result= str(searchphrase + " is not on Netflix")
                writer.writerow([searchphrase, new_list[0], video,"Failed"])
        print(result)
        return result



def amazonsearch(searchphrase):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--window-size=1920x1080")
        chrome_options.add_argument("--enable-automation")

        driver = webdriver.Chrome(chrome_options=chrome_options, executable_path='chromedriver.exe')
        driver.get("https://www.amazon.co.uk/Amazon-Video/b?&node=3010085031")
        element1 = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
        element1.send_keys(searchphrase)
        submit = driver.find_element_by_xpath('//*[@id="nav-search"]/form/div[2]/div/input')
        submit.click()

        videotitle = driver.find_elements_by_xpath('//*[@id="result_0"]/div/div/div/div[2]/div[1]/div/a/h2')
        video = ""
        for ii in videotitle:
            video = ii.get_attribute('data-attribute')

        videourls= ""
        videourl = driver.find_elements_by_xpath('//*[@id="result_0"]/div/div/div/div[2]/div[1]/div/a')
        for ii in videourl:
            videourls = ii.get_attribute('href')

        line = ""    
        Stopcondition = driver.find_elements_by_xpath('//*[@id="result_0"]/div/div/div/div[2]/div[2]/div[1]/div[1]/a/h3')
        for ii in Stopcondition:
            line = ii.get_attribute('data-attribute')
        
        driver.close()   
        with open("amazonresults.csv", "a") as csv_file1:
            writer = csv.writer(csv_file1)
            if len(line) == 0:
                newVideo = string.capwords(video)
                if searchphrase in newVideo:
                    result = searchphrase + " is on Amazon Instant Video" + "\n" + "Here is your link\n" + videourls + "\n"
                    writer.writerow([searchphrase, newVideo, videourls,"NULL","Sucessfully Found"])
                else:
                    result = str(searchphrase + " is not on Amazon Instant Video\n")
                    writer.writerow([searchphrase, newVideo, videourls ,"NULL","Failure to Find"])
            else:
                result = str(searchphrase + " is not on Amazon Instant Video\n")
                writer.writerow([searchphrase, video, videourls, line,"Failed"])

        print(result)
        return result

root = Tk()   
root.title("Find Your Media")
root.resizable(0,0) 
label1 = Label(text = "Hello Ready to find your media?!")
label2 = Label(text = "What do you want to watch?")
label3 = Label(text = "Is it avaliable?!")
button1=Button(text="GET SEARCHIN!", width=20)
entry = Text(width=50,height=3,wrap=WORD)
output = Text(width = 50, height=3, wrap=WORD)

label1.grid(row=1, column=1, padx=5, sticky=W)
label2.grid(row=2, column=1,padx=5,sticky=W)
label3.grid(row=4, column=1, padx=5, sticky=W)
entry.grid(row=3,column=1, columnspan=2,padx=5,pady=(0,10))
button1.grid(row=4,column=1,columnspan=2,pady=5)
output.grid(row=6,column=1,columnspan=2,padx=5,pady=(0,10))

button1.configure(command=resultsDisplay)
menu = Menu(root)
root.config(menu=menu)

subMenu = Menu(menu)
menu.add_cascade(label="Menu", menu=subMenu)
subMenu.add_command(label="Netflix Login", command=NetflixLogin)
subMenu.add_command(label="Netflix Table", command=netflix_window)
subMenu.add_command(label="Amazon Table", command=amazon_window)
subMenu.add_command(label="Exit", command=root.destroy)

root.mainloop()

Star Trek is on Amazon Instant Video
Here is your link
https://www.amazon.co.uk/Star-Trek-Beyond-John-Cho/dp/B01IQH3JQK/ref=sr_1_1?s=instant-video&ie=UTF8&qid=1524825115&sr=1-1&keywords=Star+Trek

